In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore") # who likes warnings right?

C:\Users\felip\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\felip\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\felip\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [8]:
def clean_data(filename):
    """@args [filename] raw csv aggregated on player per team per match, to be converted on team per match"""
    """@returns clean df with relevant features"""
    
    df = pd.read_csv(filename)
    df = df.fillna(0) # replace "empty" cells with 0
    
    relevant_features = ["MATCH", "TEAM", "PC ", "PA ", "PC/PA", "GS", "S", "SG", "FC", "FS", "DC", "D_IN_POSS",
                         "Offsides", "D_NOT_IN_POSS", "Yellow_cards", "Red_card", "A_H", "A_L", "SPRINT", "T_OPP_HALF",
                         "T_ATT_3RD"]
    df = df[relevant_features]
    
    # Fixing some minor formatting
    df["DC"] = df["DC"] / 1000 # distance from m to km
    
    # How to aggregate the features (mostly sum value for each player)
    agg_method = {feat:"sum" for feat in relevant_features}
    
    # Fix features that need the mean value of all players, not the sum of each player
    agg_method["PC/PA"] = "mean" # pass accuracy
    agg_method["A_H"] = "mean" # time in high intensity (%)
    agg_method["A_L"] = "mean" # time in high intensity (%)
    agg_method["T_OPP_HALF"] = "mean" # time spent in opponent half (%)
    agg_method["T_ATT_3RD"] = "mean" # time spent in opponent third (near box) (%)
    
    # Index values just need the first because they're all the same
    agg_method["MATCH"] = "first"
    agg_method["TEAM"] = "first"
    
    # Perform aggregation
    df = df.groupby(["MATCH", "TEAM"]).aggregate(agg_method)
    
    # Create new features and fix values
    df["goals_against"] = 0 # to be filled inside the loop below
    df["WDL"] = 0 # first assume all games tied. fixed in the loop below if needed

    for i in range(1, 65): # for each of the 64 matches
        df.loc[i, :]["goals_against"][0] = df.loc[i, :]["GS"][1] # team_0 suffers what team_2 scores
        df.loc[i, :]["goals_against"][1] = df.loc[i, :]["GS"][0] 
        if df.loc[i, :]["GS"][0] > df.loc[i, :]["GS"][1]: # if team_1 scored more than team_2
            df.loc[i, :]["WDL"][0] = 1
            df.loc[i, :]["WDL"][1] = -1
        elif df.loc[i, :]["GS"][0] < df.loc[i, :]["GS"][1]:
            df.loc[i, :]["WDL"][0] = -1
            df.loc[i, :]["WDL"][1] = 1

    df = df.rename(columns={
        "MATCH":"match",
        "TEAM":"team",
        "PC ":"passes completed",
        "PA ":"total passes",
        "PC/PA":"passes acc",
        "GS":"goals for",
        "S":"total shots",
        "SG":"on-target",
        "FC":"fouls committed",
        "FS":"fouls suffered",
        "DC":"distance",
        "D_IN_POSS":"distance poss",
        "D_NOT_IN_POSS":"distance not poss",
        "Offsides":"offsides",
        "Yellow_cards":"yellow",
        "Red_card":"red",
        "A_H":"high intensity",
        "A_L":"low intensity",
        "SPRINT":"sprints",
        "T_OPP_HALF":"time opp half",
        "T_ATT_3RD":"time opp third"
    }) # missing corners and posession, need to get those manually!
    
    df["year"] = int(filename.split("_")[1].split(".csv")[0]) # gets just "2018" or "2014"

    return df

In [9]:
games_18 = clean_data("raw_2018.csv")
games_18.head()

match team  passes completed  total passes  passes acc  goals for  \
MATCH TEAM                                                                      
1     BRA       1  BRA               433           566   67.785714        3.0   
      CRO       1  CRO               284           410   67.076923        1.0   
2     CMR       2  CMR               284           391   70.384615        0.0   
      MEX       2  MEX               484           598   74.642857        1.0   
3     ESP       3  ESP               566           680   80.642857        1.0   

            total shots  on-target  fouls committed  fouls suffered  ...   \
MATCH TEAM                                                           ...    
1     BRA          14.0        9.0              5.0            20.0  ...    
      CRO          10.0        4.0             21.0             5.0  ...    
2     CMR          10.0        4.0             12.0             9.0  ...    
      MEX           9.0        5.0             11.0            11.0  ...    
3     ESP           9.0        6.0              5.0            17.0  ...    

            yellow  red  high intensity  low intensity  sprints  \
MATCH TEAM                                                        
1     BRA      2.0  0.0        9.571429      83.285714    329.0   
      CRO      2.0  0.0        9.461538      82.000000    334.0   
2     CMR      1.0  0.0        7.538462      85.923077    318.0   
      MEX      1.0  0.0        9.357143      83.000000    320.0   
3     ESP      0.0  0.0        8.071429      84.214286    340.0   

            time opp half  time opp third  goals_against  WDL  year  
MATCH TEAM                                                           
1     BRA       49.142857       27.928571              1    1  2018  
      CRO       33.461538       15.076923              3   -1  2018  
2     CMR       41.615385       19.615385              1   -1  2018  
      MEX       44.071429       18.000000              0    1  2018  
3     ESP       47.000000       19.071429              5   -1  2018  

[5 rows x 24 columns]

In [10]:
games_14 = clean_data("raw_2014.csv")
games_14.head()

match team  passes completed  total passes  passes acc  goals for  \
MATCH TEAM                                                                      
1     BRA       1  BRA               433           566   67.785714        3.0   
      CRO       1  CRO               284           410   67.076923        1.0   
2     CMR       2  CMR               284           391   70.384615        0.0   
      MEX       2  MEX               484           598   74.642857        1.0   
3     ESP       3  ESP               566           680   80.642857        1.0   

            total shots  on-target  fouls committed  fouls suffered  ...   \
MATCH TEAM                                                           ...    
1     BRA          14.0        9.0              5.0            20.0  ...    
      CRO          10.0        4.0             21.0             5.0  ...    
2     CMR          10.0        4.0             12.0             9.0  ...    
      MEX           9.0        5.0             11.0            11.0  ...    
3     ESP           9.0        6.0              5.0            17.0  ...    

            yellow  red  high intensity  low intensity  sprints  \
MATCH TEAM                                                        
1     BRA      2.0  0.0        9.571429      83.285714    329.0   
      CRO      2.0  0.0        9.461538      82.000000    334.0   
2     CMR      1.0  0.0        7.538462      85.923077    318.0   
      MEX      1.0  0.0        9.357143      83.000000    320.0   
3     ESP      0.0  0.0        8.071429      84.214286    340.0   

            time opp half  time opp third  goals_against  WDL  year  
MATCH TEAM                                                           
1     BRA       49.142857       27.928571              1    1  2014  
      CRO       33.461538       15.076923              3   -1  2014  
2     CMR       41.615385       19.615385              1   -1  2014  
      MEX       44.071429       18.000000              0    1  2014  
3     ESP       47.000000       19.071429              5   -1  2014  

[5 rows x 24 columns]

In [11]:
games_10 = clean_data("raw_2010.csv")
games_10.head()

match team  passes completed  total passes  passes acc  goals for  \
MATCH TEAM                                                                      
1     MEX       1  MEX               447           561   75.285714          1   
      RSA       1  RSA               209           335   64.538462          1   
2     FRA       2  FRA               369           531   69.785714          0   
      URU       2  URU               277           434   54.071429          0   
3     ARG       3  ARG               446           597   73.142857          1   

            total shots  on-target  fouls committed  fouls suffered  ...   \
MATCH TEAM                                                           ...    
1     MEX            14          5               13              17  ...    
      RSA             9          5               17              11  ...    
2     FRA            18          3               20              13  ...    
      URU             7          3               13              20  ...    
3     ARG            20          7                7               8  ...    

            yellow  red  high intensity  low intensity  sprints  \
MATCH TEAM                                                        
1     MEX      0.0  0.0       10.428571      80.928571   1374.0   
      RSA      0.0  0.0       12.076923      79.923077   1690.0   
2     FRA      3.0  0.0        8.928571      83.142857   1003.0   
      URU      4.0  1.0        9.500000      82.857143    926.0   
3     ARG      0.0  0.0        7.642857      85.500000    851.0   

            time opp half  time opp third  goals_against  WDL  year  
MATCH TEAM                                                           
1     MEX       56.285714       28.642857              1    0  2010  
      RSA       27.846154        9.692308              1    0  2010  
2     FRA       56.214286       28.071429              0    0  2010  
      URU       30.214286        9.857143              0    0  2010  
3     ARG       45.142857       20.714286              0    1  2010  

[5 rows x 24 columns]

In [13]:
games_combined = pd.concat([games_18, games_14, games_10])
print(f"Total Games: {games_combined.shape[0]}")
print(f"Total Relevant Features: {games_combined.shape[1] - 4}") # remove "match", "team", "year", and WDL labels
print(f"Features: {games_combined.columns}")
games_combined.head()

Total Games: 384
Total Relevant Features: 20
Features: Index(['match', 'team', 'passes completed', 'total passes', 'passes acc',
       'goals for', 'total shots', 'on-target', 'fouls committed',
       'fouls suffered', 'distance', 'distance poss', 'offsides',
       'distance not poss', 'yellow', 'red', 'high intensity', 'low intensity',
       'sprints', 'time opp half', 'time opp third', 'goals_against', 'WDL',
       'year'],
      dtype='object')


match team  passes completed  total passes  passes acc  goals for  \
MATCH TEAM                                                                      
1     BRA       1  BRA               433           566   67.785714        3.0   
      CRO       1  CRO               284           410   67.076923        1.0   
2     CMR       2  CMR               284           391   70.384615        0.0   
      MEX       2  MEX               484           598   74.642857        1.0   
3     ESP       3  ESP               566           680   80.642857        1.0   

            total shots  on-target  fouls committed  fouls suffered  ...   \
MATCH TEAM                                                           ...    
1     BRA          14.0        9.0              5.0            20.0  ...    
      CRO          10.0        4.0             21.0             5.0  ...    
2     CMR          10.0        4.0             12.0             9.0  ...    
      MEX           9.0        5.0             11.0            11.0  ...    
3     ESP           9.0        6.0              5.0            17.0  ...    

            yellow  red  high intensity  low intensity  sprints  \
MATCH TEAM                                                        
1     BRA      2.0  0.0        9.571429      83.285714    329.0   
      CRO      2.0  0.0        9.461538      82.000000    334.0   
2     CMR      1.0  0.0        7.538462      85.923077    318.0   
      MEX      1.0  0.0        9.357143      83.000000    320.0   
3     ESP      0.0  0.0        8.071429      84.214286    340.0   

            time opp half  time opp third  goals_against  WDL  year  
MATCH TEAM                                                           
1     BRA       49.142857       27.928571              1    1  2018  
      CRO       33.461538       15.076923              3   -1  2018  
2     CMR       41.615385       19.615385              1   -1  2018  
      MEX       44.071429       18.000000              0    1  2018  
3     ESP       47.000000       19.071429              5   -1  2018  

[5 rows x 24 columns]